In [ ]:
import time
import requests


In [ ]:

# 配置
videos = [
    {"courseid": "c3469555f6fe4185b59763588d3babfd", "coursewareid": "4e84a336123b4ce1ba4c3b2bc759a05b", "duration": 300},  # 视频1，时长300秒
    {"courseid": "next_courseid", "coursewareid": "next_coursewareid", "duration": 400},  # 视频2，时长400秒
    # 添加更多视频
]

In [ ]:

base_url = "http://guizhou.zxjxjy.com"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    'X-Requested-With': 'XMLHttpRequest',
}
cookies = {
    'edu-s': '3f7392ab3e7846dc9284cfb4cb06c96f',
    'Hm_lvt_b21659b538990a950b60a32e93668ae4': '1721613496',
    'HMACCOUNT': '04CE47A58451A36E',
    'Hm_lpvt_b21659b538990a950b60a32e93668ae4': '1721624371',
}

def get_action(courseid, coursewareid, lesson_location):
    url = f"{base_url}/p/action/get"
    payload = {
        "courseid": courseid,
        "coursewareid": coursewareid,
        "lessonlocation": lesson_location,
    }
    response = requests.post(url, headers=headers, cookies=cookies, data=payload)
    print(f"GET Action Response: {response.status_code} - {response.text}")

def check_limit(courseid, coursewareid):
    url = f"{base_url}/play/checklimit?courseid={courseid}&cwid={coursewareid}"
    response = requests.get(url, headers=headers, cookies=cookies)
    print(f"Check Limit Response: {response.status_code} - {response.text}")

def submit_process(courseid, coursewareid, lesson_location):
    url = f"{base_url}/p/play/submitProcess"
    payload = {
        "courseId": courseid,
        "coursewareId": coursewareid,
        "lessonLocation": lesson_location,
    }
    response = requests.post(url, headers=headers, cookies=cookies, json=payload)
    print(f"Submit Process Response: {response.status_code} - {response.text}")

def watch_video(video):
    courseid = video["courseid"]
    coursewareid = video["coursewareid"]
    duration = video["duration"]
    
    lesson_location = 0
    last_get_check_time = time.time()
    last_submit_time = time.time()
    start_time = time.time()

    while lesson_location < duration:
        current_time = time.time()

        # 每15秒发送GET和checklimit请求
        if current_time - last_get_check_time >= 15:
            get_action(courseid, coursewareid, lesson_location)
            check_limit(courseid, coursewareid)
            check_limit(courseid, coursewareid)
            last_get_check_time = current_time

        # 每30秒发送submitProcess请求
        if current_time - last_submit_time >= 30:
            lesson_location += 30  # 假设每次播放进度增加30秒
            submit_process(courseid, coursewareid, lesson_location)
            last_submit_time = current_time

        time.sleep(1)  # 每秒钟检查一次




In [ ]:
for video in videos:
    watch_video(video)